In [ ]:
!export HF_DATASETS_CACHE="/scratch/pt2295/cache/"

In [ ]:
!export CUDA_VISIBLE_DEVICES=0

In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/scratch/pt2295/cache/'

In [ ]:
import json
import random
import re
import math



file_path = "/scratch/pt2295/LlvmProject/automatic_prompt_engineer/data/bigbench-ii/hyperbaton/task.json"

with open(file_path, 'r') as json_file:
    
    data_orig = json.load(json_file)

In [ ]:
data=data_orig['examples']

In [ ]:
len(data_orig['examples'])

In [ ]:
random.shuffle(data)

# Define the proportions for train, test, and validation sets
total_samples = len(data)
train_ratio = 0.7
test_ratio = 0.15
validation_ratio = 0.15

# Calculate the sizes of each set
train_size = int(total_samples * train_ratio)
test_size = int(total_samples * test_ratio)
validation_size = int(total_samples * validation_ratio)

# Split the data into sets
train_data = data[:train_size]
test_data = data[train_size:train_size + test_size]
validation_data = data[train_size + test_size:]

In [ ]:
import json

# Save train_data to train.json
with open('data/train_hyper.json', 'w') as train_file:
    json.dump(train_data, train_file, indent=4)

# Save test_data to test.json
with open('data/test_hyper.json', 'w') as test_file:
    json.dump(test_data, test_file, indent=4)

In [ ]:
with open('data/val_hyper.json', 'w') as val_file:
    json.dump(validation_data, val_file, indent=4)

In [ ]:
import json

# Read data from train.json
with open('data/train_hyper.json', 'r') as train_file:
    train_data = json.load(train_file)

# Read data from test.json
with open('data/test_hyper.json', 'r') as test_file:
    test_data = json.load(test_file)
with open('data/val_hyper.json', 'r') as val_file:
    validation_data=json.load(val_file)


In [ ]:
len(train_data)

In [ ]:
train_data[0]

In [ ]:
len(test_data)

# Algo

In [ ]:
#UNCOMMENT THIS TO LOAD MODEL
from transformers import AutoTokenizer,AutoModelForCausalLM
import transformers
import torch

model_name = "meta-llama/Llama-2-70b-chat-hf"
custom_cache_dir = "/scratch/pt2295/cache/"
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=custom_cache_dir,use_fast=True)


model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True,cache_dir=custom_cache_dir)




In [ ]:
task_description=data_orig['description']

In [ ]:
task_type='hyperbaton'

In [ ]:
task_prefix=data_orig['task_prefix']
if task_type=='hyperbaton':
    task_prefix=task_prefix.strip()
    task_prefix+=" Choose only from the following options: 'a' or 'b'.\n"

In [ ]:
def preprocess(text):
    return

In [ ]:
import re
def extract_answer(length,line,task_type='entailment'):
    line=line[length:]
    pattern = r'<Ans>(.*?)</Ans>|Ans:\s*([\w-]+)'
    matches = re.findall(pattern, line)
    
    pattern_exp=r'<explain>([\s\S]*?)</explain>|Explaination:\s*([\w-]+)|explaination:\s*([\w-]+)'
    matches_exp = re.findall(pattern_exp, line)
    if len(matches)==0 or len(matches_exp)==0:#right now if not match return
        return -1,""
    
    matches=matches[0][0]
   
    if task_type=='entailment':
        if matches =='entailment' or matches=='Entailment' :#or matches not in ['non-entailment','Non-entailment','Non-Entailment']:
            return 1,matches_exp[0][0].strip()
    elif task_type=='hyperbaton':
        if matches[0][0]=='a' or matches[0][0]=='A':
            return 1,matches_exp[0][0]
    return 0,matches_exp[0][0].strip()
  

In [ ]:
import re
def extract_answer_test(length,line,task_type='entailment'):
    line=line[length:]
    pattern = r'<Ans>(.*?)</Ans>|Ans:\s*([\w-]+)'
    matches = re.findall(pattern, line)
    if len(matches)==0:#right now if not match return
        return -1
    
    matches=matches[0][0]
   
    if task_type=='entailment':
        if matches =='entailment' or matches=='Entailment' :#or matches not in ['non-entailment','Non-entailment','Non-Entailment']:
            return 1
    elif task_type=='hyperbaton':
        if matches[0][0]=='a' or matches[0][0]=='A':
            return 1
    return 0
  

In [ ]:
import re
def extract_confidence(length,line,task_type='entailment'):
    line=line.lower()
    line=line[length:]
    pattern = r'<conf>(.*?)</conf>|confidence score:\s*([\w-]+)'
    matches = re.findall(pattern, line)
    if len(matches)==0:
        return ""
    matches=matches[0][0]
    
    
    
    return matches
  

In [ ]:
extract_answer(0,"<Ans>b</Ans> <explain> i am a disco dancer</explain>",task_type='hyperbaton')

In [ ]:
import requests

API_URL = "https://api-inference.huggingface.co/models/meta-llama/Llama-2-70b-chat-hf"
headers = {"Authorization": "*****"}#removing api key for security

def get_api(inp,type_prompt="infer"):
    def query(payload):
        response = requests.post(API_URL, headers=headers, json=payload)
        return response.json()
    if type_prompt=="infer":
        output = query({
            "inputs": inp,
            "parameters":{"max_new_tokens":1000,"do_sample":True,"temperature":0.75,"top_p":0.9,"use_cache":True}
        })
    elif type_prompt=="summary":
        output = query({
            "inputs": inp,
            "parameters":{"max_new_tokens":4096,"do_sample":True,"temperature":0.75,"top_p":0.9}
        })
        
    else:
        output = query({
            "inputs": inp,
            "parameters":{"max_new_tokens":1000,"do_sample":True,"temperature":0.75,"top_p":0.9}
        })


    return output

In [ ]:
#HERE THERE IS THE PROVISION OF USING LOCAL LLAMA MODEL OR INFERENCE API FROM HUGGINGFACE
def get_answer_llm(user_prompt,use_api=True,type_prompt='infer'):

    sys_prompt="You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."
    prompt=f"You are a helpful assistant. <s>[INST] {user_prompt} [/INST]"
    
    if use_api==False:
   
        model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        output = model.generate(**model_inputs)

        return tokenizer.decode(output[0], skip_special_tokens=True)
    else:

        try:
            return get_api(prompt,type_prompt)[0]['generated_text']
        except:
            return ""
        
        


In [ ]:
task_description

In [ ]:
#CHNAGE CONF PROMPT ### STEP 1: RUN INFERENCE FROM LLM TO GET CORRECT AND INCORRECT ANSWERS
def run_inference_step1(prompt_t,use_api=True,task_type='entailment'):
    wrong_ans_indices={1:[],0:[]}
    not_got=[]
    temp_prompt_t=prompt_t
    pos=[]
    neg=[]
    conf_wrong_indices={}
    for i in range(len(train_data)):     
        prompt_t=temp_prompt_t.replace('<INPUT>',train_data[i]['input'])
        op=get_answer_llm(prompt_t,use_api,type_prompt="infer")

        if op=='':
            not_got.append(i)
            continue
        output_str=op
        
        
        ans,explain=extract_answer(len(prompt_t),output_str,task_type)
        
        
        #now do confidence########
        if ans==1:
            ans_conf="Entailment"
        else:
            ans_conf="Non-entailment"
        new_prompt="Previously you were given a task of ordering adjectives correctly in English sentences."+'Given input: '+'<INPUT>'+'\n'+"This is the output you gave:"+ans_conf+"\n"+"And this is the explaination you gave for this prediction:"+explain+"\n"+"Analyze your explaination and then provide a confidence score between 0 and 1, regarding how confident you are in your prediction. Put the confidence score between the tags <conf> and </conf>. The answer should necessarily be between the respective tags."
        op_conf=get_answer_llm(new_prompt,use_api,type_prompt="infer")
        conf_score=extract_confidence(len(new_prompt),op_conf,task_type)
        if conf_score=="":
            not_got.append(i)
            continue
        
        
        ##########################
       
        if ans==-1:
            not_got.append(i)
            continue
        ground_truth=1
        
        if task_type=='entailment':
            if train_data[i]['target_scores']['non-entailment']==1:
                ground_truth=0
        elif task_type=='hyperbaton':
            if train_data[i]['target_scores']['b']==1:
                ground_truth=0

        if (ans!=ground_truth):
            
            wrong_ans_indices[ground_truth].append(i)
        if ground_truth==1:
            pos.append(i)
        else:
            neg.append(i)
            
        conf_wrong_indices[i]=conf_score
        
        
        
    
        
    
    return wrong_ans_indices,not_got,pos,neg,conf_wrong_indices

In [ ]:
def extract_hint(length,line):
    line=line[length:]
    pattern = r'<hint>(.*?)</hint>'
    matches = re.findall(pattern, line)
    if len(matches)==0:
        return ""
    return matches[0]

In [ ]:
def random_sampling(wrong_answers,k=20):
    
    k=min(k,math.ceil(len(wrong_answers)))
        
    print("k values is", k)
    
    selected=random.sample(wrong_answers, k)
    return selected
    

In [ ]:
#STEP 2: RUN HINTS GENERATION FOR THE SAMPLES

def get_hints_residual_step2(wrong_answers,pos,neg,conf_wrong_indices,use_api=True):
    hints_wrong_ans={}
    
    
    #select top 3 of entailment having highest conf score
    conf_pos={}
    for i in wrong_answers[1]:
        
        conf_pos[i]=conf_wrong_indices[i]
   
    sorted_dict_pos = dict(sorted(conf_pos.items(), key=lambda item: item[1], reverse=True))
    selected_pos_answers=list(dict(list(sorted_dict_pos.items())[:3]).keys())    #random_sampling(list(wrong_answers[1]),3)
    if len(selected_pos_answers)==0:
        selected_pos_answers=random_sampling(list(pos),3)
      
    conf_neg={}
    for i in wrong_answers[0]:
        conf_neg[i]=conf_wrong_indices[i]
    sorted_dict_neg = dict(sorted(conf_neg.items(), key=lambda item: item[1], reverse=True))
    selected_neg_answers=list(dict(list(sorted_dict_neg.items())[:3]).keys())#random_sampling(list(wrong_answers[0]),3)
    if len(selected_neg_answers)==0:
        print("ZERO*********neg")
        selected_neg_answers=random_sampling(list(neg),3)
    selected_answers=selected_pos_answers+selected_neg_answers
    print("len of selected answers in hints",len(selected_answers))
    
    for idx in selected_answers:
        if task_type=='entailment':
            if train_data[idx]['target_scores']['entailment']==1:
                ans='entailment'
            else:
                ans='non-entailment'
        elif task_type=='hyperbaton':
            if train_data[idx]['target_scores']['a']==1:
                ans='a'
            else:
                ans='b'
        prompt_h='Given following task:'+task_description+'.'+'Given input: '+train_data[idx]['input']+'.'+'And its expected output: '+ans+'\n'+'List the reason or hint why its with this expected output within tag <hint> and </hint>. The hint or explaination should be necassarily between the tags.'
       
        op=get_answer_llm(prompt_h,use_api,type_prompt="hint")
        

        hint=extract_hint(len(prompt_h),op)
        
        if idx%100==0:
            print("raw output",op,len(op))
            print("****************")
            print("hint is",hint)

       
        if hint!='':
            hints_wrong_ans[idx]=hint
   
       
        
    return hints_wrong_ans
    

In [ ]:
import random


In [ ]:
#STEP 3 is sampling and is already done and step4 is summarisation of hints
def get_summarise_step4(hints_wrong,use_api=True):

    prompt_s="This is a task to "+task_description+". We have some expected input and output pairs and have asked labeler to give reason or hint for each expected output. Given following data each contains input, output and reason for the expected output, summarize a general reason for all these cases:"+'\n'
    temp_str=''
    ctr=1

    for idx in list(hints_wrong.keys()):
        
        if len(temp_str)>=4095:
            print("the length of temp str is",len(temp_str))
            break
        temp=''
        if task_type=='entailment':
            if train_data[idx]['target_scores']['entailment']==1:
                ans='entailment'
            else:
                ans='non-entailment'
        elif task_type=='hyperbaton':
            if train_data[idx]['target_scores']['a']==1:
                ans='a'
            else:
                ans='b'

        temp+='Given input: '+train_data[idx]['input']+' '+'And its expected output: '+ans+'. And the reason for the expected output: '+hints_wrong[idx]
        temp_str+=temp
   
        
    prompt_s+=temp_str+'\n'+"Give a summary of the reasons for the example output, and do not give a reason particular to the respective example. Also do not mention the number of examples nor give any reference to entities in examples in the summary directly or indirectly. Be as general as possible. The summarised reasons are:"


    op=get_answer_llm(prompt_s,use_api,type_prompt="summary")#[l+8+7:]#or extract after [/INSTR]
    index = op.find("[/INST")
    
    extracted_text = op[index + len("[/INST]"):]
    op=extracted_text
    print("summarised op isyyyyy",op)
    return op.lstrip()
        
        
        
    

In [ ]:
task_type

# Now we have to replicate all these steps for T iterations, just a for loop

In [ ]:
import time
num_iterations_T=10
succesive_prompts=[]
start=time.time()
file_path = "data/HYPER_llama_idk_conf_100finalcorrect.txt"
prompt_t=task_description+'\n'+'Given input: '+'<INPUT>'+'\n'+task_prefix+'Put your one-word answer choosing from the previously stated two options within tag <Ans> and </Ans>. The one word answer should be necassarily between the tags. Additionally, analyze the reason for your output and provide the explaination between <explain> and </explain> tags. The respective answers should necessarily be between the respective tags.'
for t in range(num_iterations_T):
    s1=time.time()
    wrong_answers,ng,pos,neg,conf_wrong_indices=run_inference_step1(prompt_t,True,task_type)
    time.sleep(5)
    print("number of wrong_ans are",len(wrong_answers[1]),len(wrong_answers[0]),len(ng))
    print('inference finished in',time.time()-s1)
    s2=time.time()
    hints_wrong=get_hints_residual_step2(wrong_answers,pos,neg,conf_wrong_indices)
    time.sleep(5)
    print("num of hints",len(hints_wrong))
    print('hints finished in',time.time()-s2)
    s3=time.time()
    summarised_prompt=get_summarise_step4(hints_wrong)
    time.sleep(5)
    print('summarisation finished in',time.time()-s3)
    final_prompt=task_description+"\n"+'Some useful hints are: '+summarised_prompt+'\n'+'Given input: '+'<INPUT>'+'\n'+task_prefix+'Put your one-word answer choosing from the previously stated two options within tag <Ans> and </Ans>. The one word answer should be necassarily between the tags. Additionally, analyze the reason for your output and provide the explaination between <explain> and </explain> tags. The respective answers should necessarily be between the respective tags.'
    succesive_prompts.append(final_prompt)
    prompt_t=final_prompt
    print(prompt_t)
    print(time.time()-start)
    with open(file_path, "a") as file:
    # Write the string to the file
        file.write(prompt_t+"\n"+"Iteration"+str(t)+"\n")
print("Time taken",time.time()-start)
    
    



In [ ]:
succesive_prompts

In [ ]:
len(succesive_prompts)

# Now test

In [ ]:
task_type

In [ ]:
task_description

In [ ]:
task_prefix

In [ ]:
len(test_data)

In [ ]:
import time
#baseline=True will give results for baseline prompt and false for confidence prompt
def test_out(successive_prompts,task_type="hyperbaton",use_api=True,check_baseline=False):
    start=time.time()
    if check_baseline==True:
        final_prompt_totest=task_description+'\n'+'Given input: '+'<INPUT>'+'\n'+task_prefix+'Put your one-word answer choosing from the previously stated two options within tag <Ans> and </Ans>. The one word answer should be necassarily between the tags.'
    else:
        final_prompt_totest=successive_prompts[-1]

    ans_accuracy=[]
    ans_balanced=[]
    wrong_ans_indices=[]
    not_got=[]

    ground_truth_list=[]
    predicted_list=[]
        
    for i in range(len(test_data)):

        prompt_t=final_prompt_totest.replace('<INPUT>',test_data[i]['input'])
        op=get_answer_llm(prompt_t,use_api,type_prompt="infer")

        if op=='':
            not_got.append(i)
            continue
        output_str=op


        ans=extract_answer_test(len(prompt_t),output_str,task_type)
        if ans==-1:
            not_got.append(i)
            continue
        ground_truth=1

        if task_type=='entailment':
            if test_data[i]['target_scores']['non-entailment']==1:
                ground_truth=0
        elif task_type=='hyperbaton':
            if test_data[i]['target_scores']['b']==1:
                ground_truth=0
        print(ans,ground_truth)
        if (ans!=ground_truth):
            wrong_ans_indices.append(i)

        ground_truth_list.append(ground_truth)
        predicted_list.append(ans)

        print("Time taken",time.time()-start)
        
        
            
    return wrong_ans_indices,not_got,ground_truth_list,predicted_list


In [ ]:
wrong_ans_indice_test,not_got_test,y_true,y_pred=test_out(succesive_prompts)

In [ ]:
len(wrong_ans_indice_test)

In [ ]:
len(not_got_test)

In [ ]:
len(test_data)

In [ ]:
from sklearn.metrics import balanced_accuracy_score,accuracy_score

In [ ]:
balanced_accuracy_score(y_true, y_pred)

In [ ]:
accuracy_score(y_true, y_pred)